# Setup

In [9]:
import pandas as pd
import numpy as np

TRAIN_DATASET_PATH = "../datasets/meds_train_alunos.csv"
TEST_DATASET_PATH = "../datasets/meds_test_alunos.csv"
BTS_DATASET = "../datasets/Bts.csv"
IMG_PATH = "imgs/"
OUTPUT_DATASET = "../datasets/train.csv"

In [111]:
df = pd.read_csv(TRAIN_DATASET_PATH)
df.head()

,ponto_id,rssi_1_1,rssi_1_2,rssi_1_3,rssi_2_1,rssi_2_2,rssi_2_3,rssi_3_1,rssi_3_2,rssi_3_3,delay_1,delay_2,delay_3,lat,lon,indoor
0,6604,-103.705000,-110.425000,-107.381667,-110.330000,-108.345000,-106.590000,-107.153333,-102.120000,-115.00,5.0,3.0,2.0,-8.055786,-34.951505,True
1,5515,-105.225000,-115.000000,-105.210000,-115.000000,-105.835000,-115.000000,-88.470000,-94.024286,-108.12,5.0,3.0,2.0,-8.055784,-34.951608,True
2,5049,-102.610000,-115.000000,-101.380000,-115.000000,-104.295000,-97.360000,-96.980000,-99.572500,-115.00,5.0,3.0,2.0,-8.055506,-34.951693,True
3,8737,-89.140000,-97.251429,-86.642857,-96.036667,-96.592500,-92.460000,-86.813333,-102.688000,-87.27,4.0,7.0,2.0,-8.050390,-34.959630,False
4,1203,-109.088333,-107.200000,-107.010000,-108.430000,-104.845714,-101.435714,-103.065000,-102.110000,-115.00,5.0,3.0,2.0,-8.055380,-34.951736,True


# Tratamento

## Removendo o id

In [112]:
df_id = df['ponto_id']
df = df.drop(['ponto_id'], axis=1)

In [113]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6775 entries, 0 to 6774
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   rssi_1_1  6775 non-null   float64
 1   rssi_1_2  6775 non-null   float64
 2   rssi_1_3  6775 non-null   float64
 3   rssi_2_1  6775 non-null   float64
 4   rssi_2_2  6775 non-null   float64
 5   rssi_2_3  6775 non-null   float64
 6   rssi_3_1  6775 non-null   float64
 7   rssi_3_2  6775 non-null   float64
 8   rssi_3_3  6775 non-null   float64
 9   delay_1   6775 non-null   float64
 10  delay_2   6775 non-null   float64
 11  delay_3   6775 non-null   float64
 12  lat       6775 non-null   float64
 13  lon       6775 non-null   float64
 14  indoor    6775 non-null   bool   
dtypes: bool(1), float64(14)
memory usage: 747.8 KB


## Removendo duplicadas

In [114]:
num_duplicatas = df.duplicated().sum()

print("Número de duplicatas no dataset:", num_duplicatas)

Número de duplicatas no dataset: 3369


In [115]:
df.drop_duplicates(inplace=True)

In [116]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3406 entries, 0 to 6774
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   rssi_1_1  3406 non-null   float64
 1   rssi_1_2  3406 non-null   float64
 2   rssi_1_3  3406 non-null   float64
 3   rssi_2_1  3406 non-null   float64
 4   rssi_2_2  3406 non-null   float64
 5   rssi_2_3  3406 non-null   float64
 6   rssi_3_1  3406 non-null   float64
 7   rssi_3_2  3406 non-null   float64
 8   rssi_3_3  3406 non-null   float64
 9   delay_1   3406 non-null   float64
 10  delay_2   3406 non-null   float64
 11  delay_3   3406 non-null   float64
 12  lat       3406 non-null   float64
 13  lon       3406 non-null   float64
 14  indoor    3406 non-null   bool   
dtypes: bool(1), float64(14)
memory usage: 402.5 KB


## Calculando as distâncias

In [117]:
# Definições iniciais mockadas
# Setando as coordenadas (latitude e longitude) das 3 estações
bts_coords = [
    [-8.04342, -34.952],  # BTS 1
    [-8.05289, -34.944672],  # BTS 2
    [-8.05532, -34.957900]  # BTS 3
]

# Adiciona no nosso df de treino 
# as distancias euclidianas de cada ponto (lat, lon) até as BTSs
def dist(df, bts_coords):
    for index, (lat, lon) in enumerate(bts_coords, start=1):
        col_name = f'bts_{index}_dist'
        df[col_name] = np.sqrt((df['lat'] - lat)**2 + (df['lon'] - lon)**2)
    return df

In [9]:
dist(df, bts_coords).to_csv(OUTPUT_DATASET)

# Possíveis melhoras

## Modificando as distâncias

In [118]:
def delay_distance(pd):
    return pd*550, (pd+1)*550

def delays_columns(row):
    return delay_distance(row['delay_1'])[0],delay_distance(row['delay_1'])[1], delay_distance(row['delay_2'])[0],delay_distance(row['delay_2'])[1], delay_distance(row['delay_3'])[0], delay_distance(row['delay_3'])[1]

In [121]:
df[['dist_min_1', 'dist_max_1','dist_min_2', 'dist_max_2','dist_min_3', 'dist_max_3']] = df.apply(lambda row: delays_columns(row), axis=1, result_type='expand')
df = df.drop(['delay_1','delay_2','delay_3'],axis=1)

In [122]:
df.head()

,rssi_1_1,rssi_1_2,rssi_1_3,rssi_2_1,rssi_2_2,rssi_2_3,rssi_3_1,rssi_3_2,rssi_3_3,lat,lon,indoor,dist_min_1,dist_max_1,dist_min_2,dist_max_2,dist_min_3,dist_max_3
0,-103.705000,-110.425000,-107.381667,-110.330000,-108.345000,-106.590000,-107.153333,-102.120000,-115.00,-8.055786,-34.951505,True,2750.0,3300.0,1650.0,2200.0,1100.0,1650.0
1,-105.225000,-115.000000,-105.210000,-115.000000,-105.835000,-115.000000,-88.470000,-94.024286,-108.12,-8.055784,-34.951608,True,2750.0,3300.0,1650.0,2200.0,1100.0,1650.0
2,-102.610000,-115.000000,-101.380000,-115.000000,-104.295000,-97.360000,-96.980000,-99.572500,-115.00,-8.055506,-34.951693,True,2750.0,3300.0,1650.0,2200.0,1100.0,1650.0
3,-89.140000,-97.251429,-86.642857,-96.036667,-96.592500,-92.460000,-86.813333,-102.688000,-87.27,-8.050390,-34.959630,False,2200.0,2750.0,3850.0,4400.0,1100.0,1650.0
4,-109.088333,-107.200000,-107.010000,-108.430000,-104.845714,-101.435714,-103.065000,-102.110000,-115.00,-8.055380,-34.951736,True,2750.0,3300.0,1650.0,2200.0,1100.0,1650.0


In [ ]:
df_teste = pd.read_csv(TEST_DATASET_PATH)
df_ids = df_teste.iloc[:,[0]]
df_teste = df_teste.drop(columns=['ponto_id'])

In [ ]:
df_teste.head()

,rssi_1_1,rssi_1_2,rssi_1_3,rssi_2_1,rssi_2_2,rssi_2_3,rssi_3_1,rssi_3_2,rssi_3_3,dist_min_1,dist_max_1,dist_min_2,dist_max_2,dist_min_3,dist_max_3
0,-115.0,-115.0,-115.0,-115.0,-115.0,-115.0,-90.93,-84.113333,-115.0,2750.0,3300.0,1650.0,2200.0,1100.0,1650.0
1,-115.0,-115.0,-115.0,-115.0,-115.0,-115.0,-90.93,-84.113333,-115.0,2750.0,3300.0,1650.0,2200.0,1100.0,1650.0
2,-115.0,-115.0,-115.0,-115.0,-115.0,-115.0,-90.93,-86.090000,-115.0,2750.0,3300.0,1650.0,2200.0,1100.0,1650.0
3,-115.0,-115.0,-115.0,-115.0,-115.0,-115.0,-90.93,-84.113333,-115.0,2750.0,3300.0,1650.0,2200.0,1100.0,1650.0
4,-115.0,-115.0,-115.0,-115.0,-115.0,-115.0,-90.93,-84.113333,-115.0,2750.0,3300.0,1650.0,2200.0,1100.0,1650.0


## Separando o treino das labels

In [125]:
from sklearn.model_selection import train_test_split
X = df.drop(columns=['lat','lon'])
X['indoor'] = X['indoor'].astype(int)
y = df[['lat','lon']]



## Normalizando os dados

In [128]:
X.head()

,rssi_1_1,rssi_1_2,rssi_1_3,rssi_2_1,rssi_2_2,rssi_2_3,rssi_3_1,rssi_3_2,rssi_3_3,indoor,dist_min_1,dist_max_1,dist_min_2,dist_max_2,dist_min_3,dist_max_3
0,-103.705000,-110.425000,-107.381667,-110.330000,-108.345000,-106.590000,-107.153333,-102.120000,-115.00,1,2750.0,3300.0,1650.0,2200.0,1100.0,1650.0
1,-105.225000,-115.000000,-105.210000,-115.000000,-105.835000,-115.000000,-88.470000,-94.024286,-108.12,1,2750.0,3300.0,1650.0,2200.0,1100.0,1650.0
2,-102.610000,-115.000000,-101.380000,-115.000000,-104.295000,-97.360000,-96.980000,-99.572500,-115.00,1,2750.0,3300.0,1650.0,2200.0,1100.0,1650.0
3,-89.140000,-97.251429,-86.642857,-96.036667,-96.592500,-92.460000,-86.813333,-102.688000,-87.27,0,2200.0,2750.0,3850.0,4400.0,1100.0,1650.0
4,-109.088333,-107.200000,-107.010000,-108.430000,-104.845714,-101.435714,-103.065000,-102.110000,-115.00,1,2750.0,3300.0,1650.0,2200.0,1100.0,1650.0


In [130]:
from sklearn.preprocessing import StandardScaler

col_to_move = X['indoor']  # Extrai a coluna que deseja mover
X_ = X.drop(columns=['indoor'])  # Remove a coluna do DataFrame

# Adiciona a coluna de volta ao DataFrame no final


sc = StandardScaler()
X_train_names = X_.columns
X_std_train = sc.fit_transform(X_)

X_std_train = pd.DataFrame(X_std_train, columns=X_train_names)
X_std_train.reset_index(drop=True, inplace=True)
col_to_move.reset_index(drop=True, inplace=True)
y.reset_index(drop=True,inplace=True)


X_std_train = pd.concat([X_std_train, col_to_move], axis=1)
print(X_std_train.shape)
X_std_train = pd.concat([X_std_train, y], axis=1)


(3406, 16)


In [132]:
X_std_train.head()

,rssi_1_1,rssi_1_2,rssi_1_3,rssi_2_1,rssi_2_2,rssi_2_3,rssi_3_1,rssi_3_2,rssi_3_3,dist_min_1,dist_max_1,dist_min_2,dist_max_2,dist_min_3,dist_max_3,indoor,lat,lon
0,-0.905471,-1.086213,-1.318039,-0.988866,-1.698103,-1.545878,-1.893147,-1.057385,-1.153863,0.529568,0.529568,-0.225214,-0.225214,-0.50175,-0.50175,1,-8.055786,-34.951505
1,-1.058857,-1.525950,-1.093259,-1.309425,-1.410700,-2.484693,0.338785,-0.137486,-0.590159,0.529568,0.529568,-0.225214,-0.225214,-0.50175,-0.50175,1,-8.055784,-34.951608
2,-0.794973,-1.525950,-0.696830,-1.309425,-1.234365,-0.515525,-0.677829,-0.767918,-1.153863,0.529568,0.529568,-0.225214,-0.225214,-0.50175,-0.50175,1,-8.055506,-34.951693
3,0.564307,0.179995,0.828553,-0.007743,-0.352404,0.031466,0.536692,-1.121926,1.118159,-0.300869,-0.300869,2.618453,2.618453,-0.50175,-0.50175,0,-8.050390,-34.959630
4,-1.448712,-0.776235,-1.279569,-0.858446,-1.297423,-0.970501,-1.404750,-1.056249,-1.153863,0.529568,0.529568,-0.225214,-0.225214,-0.50175,-0.50175,1,-8.055380,-34.951736


In [ ]:
X_std_train.to_csv('nodelay_train.csv',index=False,header=True)

## Aplicando para o conjunto de testes

In [139]:
df_teste_names = df_teste.columns
df_teste_std = sc.transform(df_teste)
df_teste_std = pd.DataFrame(df_teste_std, columns=df_teste_names)
df_teste_std = pd.concat([df_teste_std,df_ids],axis=1)

In [141]:
df_teste_std.head()

,rssi_1_1,rssi_1_2,rssi_1_3,rssi_2_1,rssi_2_2,rssi_2_3,rssi_3_1,rssi_3_2,rssi_3_3,dist_min_1,dist_max_1,dist_min_2,dist_max_2,dist_min_3,dist_max_3,ponto_id
0,-2.045268,-1.52595,-2.106583,-1.309425,-2.460121,-2.484693,0.044911,0.988675,-1.153863,0.529568,0.529568,-0.225214,-0.225214,-0.50175,-0.50175,0
1,-2.045268,-1.52595,-2.106583,-1.309425,-2.460121,-2.484693,0.044911,0.988675,-1.153863,0.529568,0.529568,-0.225214,-0.225214,-0.50175,-0.50175,3
2,-2.045268,-1.52595,-2.106583,-1.309425,-2.460121,-2.484693,0.044911,0.764071,-1.153863,0.529568,0.529568,-0.225214,-0.225214,-0.50175,-0.50175,8
3,-2.045268,-1.52595,-2.106583,-1.309425,-2.460121,-2.484693,0.044911,0.988675,-1.153863,0.529568,0.529568,-0.225214,-0.225214,-0.50175,-0.50175,10
4,-2.045268,-1.52595,-2.106583,-1.309425,-2.460121,-2.484693,0.044911,0.988675,-1.153863,0.529568,0.529568,-0.225214,-0.225214,-0.50175,-0.50175,12


In [142]:
df_teste_std.to_csv('nodelay_teste.csv',index=False,header=True)